امه<div style="color:red;font-size:40px;">
<center>
<font size="6" >
به نام خدا 
<br><br>
<font size="5" color="grey" >
<b>
بررسی و مقایسه متن ‌نامه‌های رئیس‌جمهوران ایالات متحده آمریکا 
</b>
<br>
<font size="4" color="black">
دانشجو: سایه جارالهی 
<br>
شماره دانشجویی: 98101339
<br>
استاد: دکتر احسان‌الدین عسگری
</center>
<br>
</div>

<div dir="rtl" >
در این تمرین، داده‌های مربوط به نامه‌های رئیس جمهورهای اخیر آمریکا با استفاده از Crawling جمع آوری میشود. 
پس از آن این داده ها تمیز شده و تحلیل میشود. 
</div>

<div dir=ltr> 
<font size="6">
<b>
Phase 0: Installing libraries
</b>
<br>
</font>
</div> 

In [1]:
!pip install scrapy 
!pip install crochet
!pip install numpy 
!pip install wordcloud
import scrapy
from datetime import datetime
import re
import numpy as np
import json 
import random
import tqdm
import nltk, re
import string
from nltk.corpus import brown
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.probability import FreqDist
import itertools

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

     |████████████████████████████████| 264 kB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 57.2 MB/s 
     |████████████████████████████████| 4.0 MB 18.9 MB/s 
     |████████████████████████████████| 47 kB 4.2 MB/s 
     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 251 kB 48.6 MB/s 
     |████████████████████████████████| 87 kB 4.4 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11516 sha256=00819d2edba47a1f6e932e2e4ab3c8f31e7f991cb1530d31865c36532927aba7
  Stored in directory: /root/.cache/pip/wheels/2d/18/21/3c6a732eaa69a339198e08bb63b7da2c45933a3428b29ec454
Successfully built PyDispatcher
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package

True

<div dir=rtl> 
<font size="6">
<b>
Phase 1: Gathering Data
</b>
</font>
</div>

<div dir=rtl> 
<font size="6">
<br>
</font>
به‌ عنوان اولین گام، متن نامه‌های سه رئیس‌جمهور آمریکا استخراج میشود. این سه رئیس جمهور دونالد ترامپ و باراک اوباما و جورج بوش هستند. علت انتخاب این سه فرد، تمام شدن دوره ریاست جمهوری آن هاست و این یعنی به ازای 4 سال داده از آن ها وجود دارد. 

[از این لینک](https://www.presidency.ucsb.edu/documents/app-categories/presidential/letters)
متن نامه‌های هر رییس جمهور استخراج شده است. 
</div> 

In [2]:
class PresidentsData(scrapy.Spider):
    name = "presidents"
    presidents = ['Donald J. Trump', 'Barack Obama', 'George W. Bush']
    start_urls = [
                  "https://www.presidency.ucsb.edu/documents/presidential-documents-archive-guidebook/news-conferences-starting-with-hoover-1929"
    ]


    def parse(self, response):
        data = response.xpath('//*[@id="block-system-main"]/div/div/div/div[2]/div/div[3]/div/table/tbody/tr')
        for row in data:
            href = row.xpath('.//td[3]/a/@href').get()
            yield scrapy.Request(href, callback=self.get_transcript_body)
        next_href = response.css('li.next > a::attr(href)').get()
        if next_href and self.is_page_valid(response.text, next_href):
            a = True
            print('next page...')
            yield scrapy.Request(next_href, callback = self.parse)
        else:
            return 

    def is_page_valid(self, text, href):
        for president in self.presidents:
            if president in text:
                return True
        return False


    def get_transcript_body(self, response):
        text = response.xpath('//*[@id="block-system-main"]/div/div/div[1]/div[3]/p/text()').getall()
        body = ''
        president = response.xpath('//*[@id="block-system-main"]/div/div/div[1]/div[1]/div/div[2]/h3/a/text()').get()
        if president not in self.presidents:
            return 
        date = response.xpath('//*[@id="block-system-main"]/div/div/div[1]/div[2]/span/text()').get()
        for paragraph in text:
            if not re.match('^Q\. .*$', paragraph):
                if paragraph.startswith('The President.'):
                    paragraph = paragraph[16:]
                body += paragraph
        DATA.append({'president':president,
                     'date':date,
                     'content':body
                     })



In [3]:
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
setup()
DATA = []

class PresidentsData(scrapy.Spider):
    name = "presidents"
    presidents = ['Donald J. Trump', 'Barack Obama', 'George W. Bush']
    start_urls = [
                  'https://www.presidency.ucsb.edu/documents/app-categories/presidential/letters?page=6'
    ]


    def parse(self, response):
        data = response.css('div.field-title > p > a::attr(href)').getall()
        for index, row in enumerate(data):
            href = row
            yield scrapy.Request(response.urljoin(href), callback=self.get_transcript_body)
        next_href = response.css('li.next > a::attr(href)').get()
        if next_href and self.is_page_valid(response.text, next_href):
            a = True
            print('next page...')
            yield scrapy.Request(response.urljoin(next_href), callback = self.parse)
        else:
            return 

    def is_page_valid(self, text, href):
        for president in self.presidents:
            if president in text:
                return True
        return False


    def get_transcript_body(self, response):
        text = response.css('div.field-docs-content > p::text').getall()
        body = ''
        president = response.css('h3.diet-title > a::text').get()
        if president not in self.presidents:
            return 
        date = response.xpath('//*[@id="block-system-main"]/div/div/div[1]/div[2]/span/text()').get()
        for paragraph in text:
            if paragraph.lower() != president.lower() and not paragraph.startswith('Dear'):
                body += paragraph
        DATA.append({'president':president,
                     'date':date,
                     'content':body
                     })



@wait_for(100)
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(PresidentsData)
    return d



In [ ]:
#uncomment to crawl data
run_spider()
data = sorted(DATA, key=lambda d:datetime.strptime(d['date'], '%B %d, %Y'))
with open('data.json','w') as ff:
    ff.write(json.dumps(data))
    

In [10]:
def load_data():
    with open('data.json','r') as ff:
        data = json.load(ff)
load_data()


In [11]:
letters = {'trump':[],'obama':[], 'bush':[]}
for row in data:
    if row['president'] == 'Donald J. Trump':
        letters['trump'].append(row)
    elif row['president'] == 'Barack Obama':
        letters['obama'].append(row)
    else:
        letters['bush'].append(row)


<div dir="rtl">
پس از اجرای دو خانه بالا، داده‌ها به صورت فایل جیسون و با حجم 8 مگ در پوشه اصلی پروژه ایجاد می‌شود. در ادامه از این داده‌ها برای به دست آوردن اطلاعات استفاده میکنیم. توجه داریم که امکان لود کردن دیتا از دیتاست نیز وجود دارد.
<br> 
هریک از سطرهای داده دارای سه اطلاع نام رئیس جمهور، تاریخ نامه و متن نامه هستند. 
<br>
تعداد جزئی از داده ‌ها به صورت زیر نمایش داده شده است. 
به دلیل جلوگیری از طولانی شدن، تنها ۱۰۰ کاراکتر اول هر مصاحبه در خروجی نمایش داده شده  است.
</div>

In [12]:
import random 
print('Trump speeches:')
for trump_sample in random.sample(letters['trump'], 2):
    print('date:', trump_sample['date'])
    print('content:', trump_sample['content'][:1000])
    print('*'*50)

print('Obama speeches:')
for obama_sample in random.sample(letters['obama'], 2):
    print('date:', obama_sample['date'])
    print('content:', obama_sample['content'][:1000])
    print('*'*50)

print('bush speeches:')
for obama_sample in random.sample(letters['bush'], 2):
    print('date:', obama_sample['date'])
    print('content:', obama_sample['content'][:1000])
    print('*'*50)


Trump speeches:
date: January 04, 2019
content: 
United States Armed Forces personnel have deployed to Libreville, Gabon, to be in position to support the security of United States citizens, personnel, and diplomatic facilities in Kinshasa, Democratic Republic of the Congo. This deployment of approximately 80 personnel is in response to the possibility that violent demonstrations may occur in the Democratic Republic of the Congo in reaction to the December 30, 2018, elections there. The first of these personnel arrived in Gabon on January 2, 2019, with appropriate combat equipment and supported by military aircraft. Additional forces may deploy to Gabon, the Democratic Republic of the Congo, or the Republic of the Congo, if necessary for these purposes. These deployed personnel will remain in the region until the security situation in the Democratic Republic of the Congo becomes such that their presence is no longer needed.
This action was taken consistent with my responsibility to pro

<div dir="ltr">
<font size="6">
<b>
Phase 2: Tokenization
</b>
<br>
</font>
<div dir="rtl">
<font size="6">
<b>
گام دوم: توکنایزیشن
</b>
<br>
</font>
در این بخش متن به جملات و سپس جملات به توکن کلمات شکسته می شوند. 

</div>

In [13]:
def handle_comma_space(text):
    return re.sub(r'(!"\#|\$|%|\&|\(|\)|\*|\+|,|\-|\.|/|:|;|<|=|>|\?|@|\[|\]|\^|_|`|\{|\||\}|\~)',f'\\1 ', text)


def tokenize(transcript):
    handle_comma_space(transcript)
    result = []
    sentence_token = sent_tokenize(transcript)
    for sentence in sentence_token:
        result.append(word_tokenize(sentence))
    return result

<div dir="ltr">
<font size="6">
<b>
Phase 3: Normalization</b>
<br>
</font>
</div>
<div dir="rtl">
<font size="6">
<b>
گام سوم: نرمالیزیشن
</b>
<br>
</font>
در گام بعدی، جملات و متن ها نرمالایز میشوند.
</div>

In [14]:
minimum_word_len = 2
stopwords = [x.lower() for x in nltk.corpus.stopwords.words('english')]

In [15]:
from copy import deepcopy
def remove_stop_words(sentences):
    return [[word for word in sentence if (word.lower() not in stopwords)] for sentence in sentences]

def convert_to_lower(sentences):
    return [[word.lower() for word in sentence] for sentence in sentences]   

def apply_min_len(sentences:list):
        return [[word for word in sentence if len(word)>minimum_word_len-1] for sentence in sentences]  
    
def remove_punctuations(sentences):
    return [[word for word in sentence if word not in string.punctuation] for sentence in sentences ]

def remove_all_punctuation(sentences):
    return [[word for word in sentence if is_there_non_punc(word)] for sentence in sentences]

def check_start(sentences):
    return [[word for word in sentence if not word.startswith('\'')] for sentence in sentences]


def is_there_non_punc(word):
    for c in word:
        if c not in string.punctuation:
            return True
    return False

def normalize(sentences: list):
    temp = deepcopy(sentences)
    temp = remove_stop_words(temp)
    temp = remove_punctuations(temp)
    temp = convert_to_lower(temp)
    temp = remove_all_punctuation(temp)
    temp = check_start(temp)
    temp = apply_min_len(temp)
    return temp 


<div dir="rtl"><font size="6">
<b>
بررسی عملکرد توکنایزر و نرمالایزر و مقایسه تفاوت‌های متن 
</b>
</font>
<br>
در ابتدا متن مصاحبه ها را به کمک Tokenizer ، به توکن ها میشکنیم و سپس 
آن ها را نرمالایز میکنیم. 
در انتها توکن‌ها را پیش و پس از نرمالایز کردن مقایسه میکنیم. 
</div>

In [16]:
statistics = {'obama':{'word':0, 'sent': 0, 'letter':len(letters['obama'])}, 
              'trump': {'word':0, 'sent': 0, 'letter':len(letters['trump'])},
              'bush' : {'word':0, 'sent': 0, 'letter':len(letters['bush'])}
              }
tokenized_letters = {'trump':[], 'bush':[], 'obama':[]}
normalized_letters = {'trump':[], 'bush':[], 'obama':[]}


In [17]:
## tokenizing speeches

def tokenize_letters():
    for president, p_letters in letters.items():
        for letter in p_letters:
            date = letter['date']
            text = letter['content']
            tokens = tokenize(text)
            statistics[president]['sent'] += len(tokens)
            for i in tokens:
                statistics[president]['word'] += len(i)
            tokenized_letters[president].append(dict(date=date, content=tokens))

def get_tokenized_samples():
    for president, p_letters in tokenized_letters.items():
        print('president', president)
        print(len(p_letters))
        r_index = random.randint(0, len(p_letters))
        print('tokens: ')
        print(p_letters[r_index]['content'])
        print('initial_speech:')
        print(letters[president][r_index]['content'])
        print('*'*20)

In [18]:
tokenize_letters()
get_tokenized_samples()

president trump
205
tokens: 
[['In', 'accordance', 'with', 'section', '303', '(', 'a', ')', '(', '6', ')', '(', 'A', ')', 'of', 'the', 'Defense', 'Production', 'Act', 'of', '1950', ',', 'as', 'amended', '(', 'the', '``', 'Act', "''", ')', '(', '50', 'U.S.C', '.'], ['4533', '(', 'a', ')', '(', '6', ')', '(', 'A', ')', ')', ',', 'I', 'transmit', 'herewith', 'notice', 'of', 'actions', 'to', 'rectify', 'a', 'shortfall', 'in', 'the', 'defense', 'industrial', 'base', '.'], ['The', 'Department', 'of', 'Defense', 'will', 'take', 'actions', 'to', 'target', 'critical', 'technology', 'item', 'shortfalls', 'related', 'to', 'the', 'following', ':', 'thin-wall', 'castings', 'for', 'military', 'applications', '.'], ['I', 'have', 'also', 'included', 'my', 'determination', 'required', 'pursuant', 'to', 'section', '303', '(', 'a', ')', '(', '5', ')', 'of', 'the', 'Act', '.'], ['Please', 'find', 'my', 'determination', 'that', 'these', 'proposed', 'initiatives', 'are', 'essential', 'to', 'the', 'national'

In [19]:
#normalize letters
def normalize_letters():
    for president, p_letters in tokenized_letters.items():
        for letter in p_letters:
            date = letter['date']
            tokens = letter['content']
            normalized = normalize(tokens)
            normalized_letters[president].append(dict(date=date, content= normalized))


In [21]:
normalize_letters()

<div dir="rtl">
<font size="3">
حال که توکن ها را به دست آوردیم و نرمال سازی را نیز انجام دادیم، چند نمونه از تفاوت‌های توکن‌ها قبل و بعد از نرمال شدن را بررسی میکنیم. 

</div>

In [22]:
def print_data(normalized, tokenized):
    print('normalized version:')
    print(normalized)
    print('only tokenized version:')
    print(tokenized)

for president in letters.keys():
    print('president', president,':')
    for _ in range(2):
        rand_i = random.randint(0, len(normalized_letters[president]))
        normalized = normalized_letters[president][rand_i]['content']
        tokenized = tokenized_letters[president][rand_i]['content']
        print_data(normalized, tokenized)
    print('*'*20)

president trump :
normalized version:
[['accordance', 'section', '9021', 'consolidated', 'appropriations', 'act', '2018', 'public', 'law', '115-141', 'transmit', 'herewith', 'report', 'united', 'states', 'strategy', 'defeat', 'islamic', 'state', 'iraq', 'syria', 'isis', 'includes', 'analysis', 'budgetary', 'resources', 'necessary', 'accomplish', 'strategy'], ['secretary', 'defense', 'consultation', 'secretary', 'state', 'director', 'national', 'intelligence', 'previously', 'provided', 'report', 'congress', 'june', '23', '2017'], ['addition', 'consistent', 'section', '9021', 'consolidated', 'appropriations', 'act', '2018', 'also', 'transmit', 'herewith', 'report', 'legal', 'policy', 'frameworks', 'guiding', 'united', 'states', 'use', 'military', 'force', 'related', 'national', 'security', 'operations'], ['report', 'previously', 'provided', 'congress', 'march', '12', '2018.sincerely']]
only tokenized version:
[['In', 'accordance', 'with', 'section', '9021', 'of', 'the', 'Consolidated', '

<div dir="rtl">
<font size="3">
همانطور که از خروجی ها واضح است، پس از نرمالایز کردن کلماتی که باقی میمانند دارای معنای بیشتری هستند و حروف اضافه و علائم نگارشی و ... را شامل نمیشوند.
</div>

<div dir="rtl"><font size="6">
<b>
بررسی حجم داده ورودی </b>
</font>
<br>
تعداد جملات و توکن‌های کلمه عریک از دو رئیس جمهور به تفکیک به دست آورده شده است: 
</div>

In [23]:
for president, stats in statistics.items():
    print(f'{president} data:')
    print('total letters', stats['letter'])
    print('total sentences:', stats['sent'])
    print('total words:', stats['word'])
    print('average words per letter:', stats['word']/ stats['letter'])
    print('average sentence per letter:', stats['sent']/stats['letter'])
    print('*'*20)

obama data:
total letters 297
total sentences: 2538
total words: 106534
average words per letter: 358.7003367003367
average sentence per letter: 8.545454545454545
********************
trump data:
total letters 205
total sentences: 1885
total words: 59554
average words per letter: 290.5073170731707
average sentence per letter: 9.195121951219512
********************
bush data:
total letters 430
total sentences: 2374
total words: 100366
average words per letter: 233.4093023255814
average sentence per letter: 5.52093023255814
********************


<div dir="rtl">
همانطور که از داده ها معلوم است، نامه‎‌های اداری 
بوش بسیار کوتاه‌تر و خلاصه‌تر بوده است. 
</div>

<div dir="ltr"><font size="6">
<b>
Phase4: Get the frequency of each word in different texts after normalization and compare with the results before normalization 
</b>
</font>
<br>
</div>
<div dir="rtl"><font size="6">
<b>
گام چهارم: بررسی بسامد کلمات در متن‌ها پس از نرمال‌سازی و مقایسه آن با زمان قبل از نرمال‌سازی
</b>
</font>
<br>
</div>

In [24]:
def get_words(sentences:list):
    words = []
    for sent in sentences:
        words.extend(sent)
    return words

def get_sentences(letter):
    sents = []
    for s in letter:
        sents.extend(s)
    return sents

In [25]:
def get_frequency(letters, count):
    words = get_words(get_sentences(letters))
    freqdist = FreqDist(words)            
    top=freqdist.most_common(count)      
    print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
    for topword in top:
        percent=(topword[1]/len(words))*100
        print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

In [26]:
# getting frequency differences
for president in letters.keys():
    print(president, 'top tokenized words:')
    get_frequency([x['content'] for x in tokenized_letters[president]],10)
    print(president, 'top normalized words:')
    get_frequency([x['content'] for x in normalized_letters[president]], 10)
    print('********************')

trump top tokenized words:
word             Frequency        % of the total  
the              3735             6.271619034825536
,                3498             5.873660879202069
of               2446             4.1071968297679415
to               2121             3.5614736205796422
and              1929             3.239077140074554
.                1704             2.8612687644826544
in               1247             2.0938979749471067
(                951              1.596870067501763
)                951              1.596870067501763
United           646              1.0847298250327435
trump top normalized words:
word             Frequency        % of the total  
united           648              2.144985104270109
states           640              2.118503806686528
national         475              1.5723270440251573
act              370              1.2247600132406489
emergency        368              1.2181396888447533
order            352              1.1651770936775903
fo

<div dir = "rtl">
<br><br>
حال stopwords ها را 
از کلمات حذف میکنیم. این کار را در بخش normalization نکردیم تا در اینجا بتوانیم
تفاوت در فرکانس را مشاهده کنیم. 
</div>

In [27]:
stop_word_removed_letter = {'trump':[], 'obama':[], 'bush':[]}

def remove_stopwords(sents):
    stop_words = nltk.corpus.stopwords.words('english') + ['united', 'states', 'u.s', 'america', 'u.s.']
    return [[word.lower() for word in sentence if word not in stop_words] for sentence in sents] 

for pr, pr_normalized in normalized_letters.items():
    print('president', pr)
    for letter in pr_normalized:
        content = remove_stopwords(letter['content'])
        stop_word_removed_letter[pr].append(dict(content= content, date= letter['date']))
    get_frequency([letter['content'] for letter in stop_word_removed_letter[pr]], 10)
    print('*'*20)



president trump
word             Frequency        % of the total  
national         475              1.645021645021645
act              370              1.2813852813852813
emergency        368              1.2744588744588745
order            352              1.2190476190476192
forces           318              1.1012987012987012
executive        297              1.0285714285714285
section          263              0.9108225108225109
continue         258              0.8935064935064935
security         225              0.7792207792207793
government       168              0.5818181818181818
********************
president bush
word             Frequency        % of the total  
act              560              1.0984484415762736
emergency        473              0.9277966301171024
security         467              0.9160275396716424
congress         413              0.8101057256625016
national         387              0.7591063337321747
section          377              0.7394911829897413

<div dir="ltr"><font size="6">
<b>
Phase 5: Lemmatizer
</b>
</font>
<br>
</div>

<div dir = "rtl">
توجه: در ابتدا از stemmer نیز 
استفاده کردم، اما نتایج مناسبی نداشت و در برخی از کلمات، اشتباه تبدیل میکرد و تاثیر خاصی نداشت. در نتیجه تنها lemmatizer را ذخیره کردم. 
</div>

In [28]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

lemmatized_letters = {'trump':[], 'obama':[], 'bush':[]}
for president, normalized_p in stop_word_removed_letter.items():
    for letter in normalized_p:
        content = letter['content']
        new_letter =  [[lemmatizer.lemmatize(word) for word in sentence] for sentence in content]
        lemmatized_letters[president].append(dict(content=new_letter, date=letter['date']))
    

In [29]:
for president, lemmatized_letter in lemmatized_letters.items():
    print('president', president)
    get_frequency([letter['content'] for letter in lemmatized_letter], 10)
    print('*'*20)

president trump
word             Frequency        % of the total  
national         485              1.6796536796536796
emergency        443              1.5341991341991343
act              381              1.3194805194805195
force            376              1.302164502164502
order            362              1.2536796536796537
executive        297              1.0285714285714285
section          274              0.9489177489177489
continue         258              0.8935064935064935
security         240              0.8311688311688312
government       178              0.6164502164502164
********************
president obama
word             Frequency        % of the total  
force            822              1.5474397590361446
national         682              1.2838855421686748
emergency        626              1.1784638554216866
security         554              1.042921686746988
order            547              1.0297439759036144
continue         417              0.7850150602409638

<div dir="rtl">
در این بخش باتوجه به اینکه در 10 کلمه اول هیچیک کلمات هم ریشه وجود نداشت، آمار تغییر نکرد. اگر برای خروجی های بزرگتر در نظر بگیریم، این آمار تغییر میکند. 
</div>

<div dir="ltr"><font size="6">
<b>
Phase 6: POS-Tagging 
</b>
</font>
<br>
در این بخش دو تگر brown
 و 
 nltk.pos_tagger
 استفاده شده است. خروجی این دو باهم مقایسه شده است. 
</div>

In [123]:
from nltk.corpus.reader.sentiwordnet import SentiWordNetCorpusReader
from nltk import word_tokenize, pos_tag
from nltk.corpus import brown
class Taggers:

    def __init__(self, brown_tagger = None):
        self.brown_tagger = None


    def create_data_sets(self, sentences, percentage):
        size = int(len(sentences) * percentage)
        train_sents = sentences[:size]
        test_sents = sentences[size:]
        return train_sents, test_sents

    def build_backoff_tagger (self, train_sents):
        t0 = nltk.DefaultTagger('NN')
        t1 = nltk.UnigramTagger(train_sents, backoff=t0)
        t2 = nltk.BigramTagger(train_sents, backoff=t1)
        return t2


    def train_brown_tagger(self, already_tagged_sents):
        train_sents, test_sents = self.create_data_sets(already_tagged_sents, 0.9)
        ngram_tagger = self.build_backoff_tagger(train_sents)
        accuracy = ngram_tagger.evaluate(test_sents)
        self.brown_tagger = ngram_tagger
        return ngram_tagger, accuracy  
    
    def tag_letters_by_brown(self, letters):
        s1  = letters.copy()
        s1 = s1.content
        res = []
        words = get_words(s1)
        res.append([self.brown_tagger.tag(word) for word in words])
        return res

    def tag_letters_by_nltk(self, letters):
        s1  = letters.copy()
        s1 = s1.content
        words = get_words(s1)
        res.append([self.brown_tagger.tag(word) for word in words])
        return res

    def get_words(self, letters):
        res = []
        for letter in letters:
            for sent in letter:
                res.extend(sent)
        return res


In [124]:
taggers = Taggers()
brown_tagged_sents = brown.tagged_sents(categories=brown.categories())
brown_tagger, acc =taggers.train_brown_tagger(brown_tagged_sents)
print('%0.3f pos accuracy on test set' % acc)

0.913 pos accuracy on test set


In [125]:
import pandas as pd

tagged_letters = {'trump':[], 'obama':[], 'bush': []}
for p , l in lemmatized_letters.items():
    p_letters = [l1['content'] for l1 in l]
    p_df = pd.DataFrame()
    p_df['content'] = p_letters
    res = taggers.tag_letters_by_brown(p_df)
    tagged_letters[p].append(res)

<div dir = "rtl">
حال به بررسی و نمایش بخشی از داده ها به طور رندوم میپردازیم. برای مثال یک نامه ترامپ را نگاه میکنیم. 
</div>

In [127]:
(tagged_letters['trump'][0][0][:10])


[[('recognition', 'NN'),
  ('national', 'JJ'),
  ('catholic', 'NN'),
  ('school', 'NN'),
  ('week', 'NN'),
  ('want', 'VB'),
  ('extend', 'VB'),
  ('heartfelt', 'JJ'),
  ('appreciation', 'NN'),
  ('dedicated', 'VBN'),
  ('catholic', 'NN'),
  ('school', 'NN'),
  ('administrator', 'NN'),
  ('teacher', 'NN'),
  ('priest', 'NN'),
  ('support', 'NN'),
  ('organization', 'NN'),
  ('work', 'NN'),
  ('tirelessly', 'RB'),
  ('build', 'VB'),
  ('sustain', 'VB'),
  ('quality', 'NN'),
  ('catholic', 'NN'),
  ('school', 'NN'),
  ('across', 'IN'),
  ('nation', 'NN')],
 [('theme', 'NN'),
  ('year', 'NN'),
  ('national', 'JJ'),
  ('catholic', 'NN'),
  ('school', 'NN'),
  ('week', 'NN'),
  ('catholic', 'NN'),
  ('school', 'NN'),
  ('community', 'NN'),
  ('faith', 'NN'),
  ('knowledge', 'NN'),
  ('service', 'NN')],
 [('appreciate', 'VB'),
  ('many', 'AP'),
  ('way', 'NN'),
  ('catholic', 'NN'),
  ('school', 'NN'),
  ('nurture', 'VB'),
  ('devotion', 'NN'),
  ('impart', 'VB'),
  ('wisdom', 'NN'),
  ('min

###Technical Terms

In [129]:
def chunkTechnicalTerm(sentence):
    grammar = r"""
      TECHTERM: {<JJ|NN>+<NN|CD>|<NN>}
    """
    cp = nltk.RegexpParser(grammar)
    return (cp.parse(sentence))

import copy

technical_terms = {}
opt_technical_terms = {}



for opt in tqdm.tqdm(['trump', 'obama', 'bush']):
    technical_terms[opt] = []
    tagged = (tagged_letters[opt][0][0])
    for sentence in tagged:
        tree=chunkTechnicalTerm(sentence)
        for subtree in tree.subtrees():
            if subtree.label() == 'TECHTERM':
                technical_terms[opt].append(subtree.leaves())

for opt in tqdm.tqdm(['obama', 'bush', 'trump']):
    opt_technical_terms[opt]=FreqDist([' '.join([x for x,y in sent]) for sent in technical_terms[opt] if len(sent)>1]).most_common(40)    
                                   
opt_technical_terms = pd.DataFrame(opt_technical_terms) 
opt_technical_terms


 33%|███▎      | 1/3 [00:00<00:00,  4.32it/s]

 67%|██████▋   | 2/3 [00:00<00:00,  2.10it/s]

100%|██████████| 3/3 [00:00<00:00, 64.63it/s]


,obama,bush,trump
0,"(national emergency, 96)",(letter director office management budget.sinc...,"(national emergency, 74)"
1,"(section 202, 91)","(national emergency act 50, 43)","(national emergency act 50, 63)"
2,"(national emergency act 50, 91)","(section 202, 41)","(section 202, 59)"
3,"(automatic termination national emergency, 91)","(automatic termination national emergency, 40)","(automatic termination national emergency, 59)"
4,"(anniversary date declaration president, 91)","(anniversary date declaration president, 40)","(congress notice, 59)"
5,"(congress notice, 91)","(congress notice, 40)","(anniversary date, 58)"
6,"(anniversary date, 91)","(anniversary date, 40)","(accordance provision, 57)"
7,"(accordance provision, 88)","(national emergency, 38)","(day anniversary date declaration president, 56)"
8,(unusual extraordinary threat national securit...,"(accordance provision, 35)","(executive order, 28)"
9,(unusual extraordinary threat national securit...,"(letter director office management budget, 32)","(congress act, 27)"


<div dir="rtl"><font size="6">
<b>
نتیجه نهایی
</b>
</font>
<br>
داده های به دست آمده در بالا، در صورتی که به دست افراد تحلیلگر برسند،‌میتوانند باعث اطلاعات بیشتر در زمینه ارتباطات رئیس جمهور ها و تمرکز انها شوند. 
برای مثال در صحبت های ترامپ بحث کلمات امنیت بیشتر آمده،. 
</div>